# 1. Checagem dos dados

Este script inicial tem como objetivo realizar checagens nas bases de dados fornecidas para a execução do case

#### Objetivo do Case | Expansão de uma rede de laboratórios
*Apresentar uma análise exploratória dos dados de uma rede de medicina diagnóstica para definir caminhos de abordagem  que auxiliem a área de negócios da empresa a ter insumos iniciais para suportar seu plano de expansão.*

### importando as bibliotecas necessárias

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

### lendo as bases de dados

In [116]:
demographic_data = pd.read_csv('dados_brutos/DemographicData_ZCTAs.csv', index_col = 0)
df_geocode = pd.read_csv('dados_brutos/df_geocode.csv', index_col = 0)
economic_data = pd.read_csv('dados_brutos/EconomicData_ZCTAs.csv', index_col = 0)
test_data = pd.read_csv('dados_brutos/test_data.csv')
transactional_data = pd.read_csv('dados_brutos/transactional_data.csv', sep = ';')

### qual o número de linhas e de colunas de cada base?

In [13]:
pd.DataFrame({
    'base': ['demographic_data', 'df_geocode', 'economic_data', 'test_data', 'transactional_data'],
    'n_colunas': [len(demographic_data.columns), len(df_geocode.columns), len(economic_data.columns), len(test_data.columns), len(transactional_data.columns)],
    'n_linhas': [len(demographic_data), len(df_geocode), len(economic_data), len(test_data), len(transactional_data)]
})

,base,n_colunas,n_linhas
0,demographic_data,18,33120
1,df_geocode,4,119
2,economic_data,12,629280
3,test_data,6,2002
4,transactional_data,8,2355241


### verificando as primeiras linhas de cada dataframe

1. demographic_data

In [12]:
# dados demográficos relevantes a nível de ZCTA (zip code tabulation area)
demographic_data.head()

,Id,GeographicAreaName,TotalPopulation,SexRatio(males per 100 females),Population_Under5Years,Population_5to9Years,Population_10to14Years,Population_15to19Years,Population_20to24Years,Population_25to34Years,Population_35to44Years,Population_45to54Years,Population_55to59Years,Population_60to64Years,Population_65to74Years,Population_75to84Years,Population_85YearsAndOver,MedianAgeInYears
0,8600000US35004,ZCTA5 35004,12045,94.1,805,1075,898,477,578,2088,1628,1200,886,683,1017,534,176,35.5
1,8600000US35005,ZCTA5 35005,7344,86.1,504,453,511,499,214,788,795,968,612,561,798,485,156,44.0
2,8600000US35006,ZCTA5 35006,2883,108.2,96,153,303,129,156,183,367,430,296,260,280,201,29,47.2
3,8600000US35007,ZCTA5 35007,26332,95.0,1936,1992,1837,1762,1376,3119,3849,3907,1665,1323,2096,1106,364,37.7
4,8600000US35010,ZCTA5 35010,20613,90.5,1306,1465,944,1217,1128,2513,2106,2950,1512,1472,2421,1155,424,42.6


2. df_geocode

In [11]:
# informações a respeito da localização de cada laboratório, como endereço, lat-long e ZCTA (zipcode) no qual o laboratório se encontra
df_geocode.head()

,Lab Id,Address,Location,Zipcode
0,L152,"3800 PLEASANT HILL RD STE 1, DULUTH, GA 30096","34.000319,-84.1629724",30096.0
1,L520,"1614 N JAMES ST, ROME, NY 13440","43.2311327,-75.4445363",13440.0
2,L141,"12911 120TH AVE NE STE D60, KIRKLAND, WA 98034","47.7162786,-122.1838152",98034.0
3,L524,"5667 PEACHTREE DUNWOODY RD 250, ATLANTA, GA 30342","33.9093875,-84.3529096",30342.0
4,L545,"1204 IL HWY 164, OQUAWKA, IL 61469","40.9309925,-90.9437598",61469.0


3. economic_data

In [26]:
# dados econômicos relevantes a nível de ZCTAs
economic_data.head()

,id,Geographic Area Name,TotalHouseholds_LessThan$10.000,TotalHouseholds_$10.000to$14.999,TotalHouseholds_$15.000to$24.999,TotalHouseholds_$25.000to$34.999,TotalHouseholds_$35.000to$49.999,TotalHouseholds_$50.000to$74.999,TotalHouseholds_$75.000to$99.999,TotalHouseholds_$100.000to$149.999,TotalHouseholds_$150.000to$199.999,TotalHouseholds_$200.000OrMore
0,8600000US35004,ZCTA5 35004,198,71,298,513,647,1117,529,945,245,61
1,8600000US35005,ZCTA5 35005,188,184,318,293,353,562,299,407,67,26
2,8600000US35006,ZCTA5 35006,71,20,117,104,154,176,124,194,51,7
3,8600000US35007,ZCTA5 35007,396,208,670,462,1173,1854,1578,2224,473,254
4,8600000US35010,ZCTA5 35010,700,610,1093,957,1056,1512,807,749,254,249


In [125]:
# a coluna "id" está em minúsculo, e a "Geographic Area Name" está com as palavras separadas.
# é importante alterar os nomes para ficarem exatamente iguais aos nomes dessas respectivas colunas no dataframe demographic_data

economic_data = economic_data.rename({'id':'Id', 'Geographic Area Name':'GeographicAreaName'}, axis = 1)
economic_data.head()

,Id,GeographicAreaName,TotalHouseholds_LessThan$10.000,TotalHouseholds_$10.000to$14.999,TotalHouseholds_$15.000to$24.999,TotalHouseholds_$25.000to$34.999,TotalHouseholds_$35.000to$49.999,TotalHouseholds_$50.000to$74.999,TotalHouseholds_$75.000to$99.999,TotalHouseholds_$100.000to$149.999,TotalHouseholds_$150.000to$199.999,TotalHouseholds_$200.000OrMore
0,8600000US35004,ZCTA5 35004,198,71,298,513,647,1117,529,945,245,61
1,8600000US35005,ZCTA5 35005,188,184,318,293,353,562,299,407,67,26
2,8600000US35006,ZCTA5 35006,71,20,117,104,154,176,124,194,51,7
3,8600000US35007,ZCTA5 35007,396,208,670,462,1173,1854,1578,2224,473,254
4,8600000US35010,ZCTA5 35010,700,610,1093,957,1056,1512,807,749,254,249


4. test_data

In [11]:
# informações a respeito dos exames, características como custo de aplicação e outras especificidades técnicas
test_data.head()

,CodItem,Desc Item,Category,Family,Speciality,Testing Cost
0,70003237,"MAGNESIO, SORO (EXEMPLO DE EXPRESSAO DE RESULT...",CA,CORELAB,BIOCHEMISTRY,1.78
1,70000638,"HEMOGRAMA, AUTOMATIZADO, SANGUE",CA,HEMATO,BLOOD COUNT,2.46
2,70001597,"FERRITINA, SORO",CA,CORELAB,IMMUNOHORMONE,2.11
3,70000103,"FERRO, DOSAGEM, SORO (EXEMPLO DE EXPRESSAO DE ...",CA,CORELAB,BIOCHEMISTRY,0.80
4,70000224,"CALCIO, DOSAGEM, SORO (EXEMPLO DE EXPRESSAO DE...",CA,CORELAB,BIOCHEMISTRY,1.02


5. transactional_data

In [7]:
# registros de exames feitos por pacientes da rede em cada laboratório
transactional_data.head()

,Patient Id,Gender,Date of birth,Date of service,Service Id,Lab Id,CodItem,Testing Cost
0,10210830256-BIO003,F,01/08/1976 00:00:00,2019-01-07,571904533475-38,L133,70003237,"9,0"
1,10210830256-BIO003,F,01/08/1976 00:00:00,2019-01-07,571904533475-38,L133,70000638,"13,0"
2,10210830256-BIO003,F,01/08/1976 00:00:00,2019-01-07,571904533475-38,L133,70001597,"49,0"
3,10210830256-BIO003,F,01/08/1976 00:00:00,2019-01-07,571904533475-38,L133,70000103,"11,0"
4,10210830256-BIO003,F,01/08/1976 00:00:00,2019-01-07,571904533475-38,L133,70000224,"10,0"


### mapeamento das colunas que relacionam as bases disponíveis

criando um dataframe com a lista de colunas de cada base

In [126]:
df1 = demographic_data.columns.tolist()
df1 = pd.DataFrame(df1, columns = ['columns'])
df1['1_demographic_data'] = 'x'

df2 = df_geocode.columns.tolist()
df2 = pd.DataFrame(df2, columns = ['columns'])
df2['2_df_geocode'] = 'x'

df3 = economic_data.columns.tolist()
df3 = pd.DataFrame(df3, columns = ['columns'])
df3['3_economic_data'] = 'x'

df4 = test_data.columns.tolist()
df4 = pd.DataFrame(df4, columns = ['columns'])
df4['4_test_data'] = 'x'

df5 = transactional_data.columns.tolist()
df5 = pd.DataFrame(df5, columns = ['columns'])
df5['5_transactional_data'] = 'x'

# exemplo de como fica o dataframe com a lista de colunas para cada base:
df2


,columns,2_df_geocode
0,Lab Id,x
1,Address,x
2,Location,x
3,Zipcode,x


juntando todas as listas criadas para verificar as colunas em comum

In [136]:
merged = df1.merge(df2, how = 'outer').merge(df3, how = 'outer').merge(df4, how = 'outer').merge(df5, how = 'outer')
merged

,columns,1_demographic_data,2_df_geocode,3_economic_data,4_test_data,5_transactional_data
0,Id,x,NaN,x,NaN,NaN
1,GeographicAreaName,x,NaN,x,NaN,NaN
2,TotalPopulation,x,NaN,NaN,NaN,NaN
3,SexRatio(males per 100 females),x,NaN,NaN,NaN,NaN
4,Population_Under5Years,x,NaN,NaN,NaN,NaN
5,Population_5to9Years,x,NaN,NaN,NaN,NaN
6,Population_10to14Years,x,NaN,NaN,NaN,NaN
7,Population_15to19Years,x,NaN,NaN,NaN,NaN
8,Population_20to24Years,x,NaN,NaN,NaN,NaN
9,Population_25to34Years,x,NaN,NaN,NaN,NaN


filtrando apenas as colunas que aparecem em pelo menos 2 bases

In [132]:
merged['n_dfs'] = merged.count(axis = 1) - 2 # conta os número de colunas com valores não-nulos, subtraindo 2 (index e coluna dos nomes)
merged[merged['n_dfs'] >= 2] # filtra apenas os nomes de colunas que aparecem em pelo menos 2 bases

,columns,1_demographic_data,2_df_geocode,3_economic_data,4_test_data,5_transactional_data,n_dfs
0,Id,x,NaN,x,NaN,NaN,2
1,GeographicAreaName,x,NaN,x,NaN,NaN,2
18,Lab Id,NaN,x,NaN,NaN,x,2
32,CodItem,NaN,NaN,NaN,x,x,2
37,Testing Cost,NaN,NaN,NaN,x,x,2


Observando o dataframe acima, é possível perceber que as bases **test_data** e **transactional_data** possuem 2 colunas em comum. 

Além disso, as bases **demographic_data** e **economic_data** também possuem 2 colunas em comum.

É importante investigar se essas duas colunas são correspondentes

### investigando as duas colunas em comum das bases test_data e transactional_data

In [94]:
test_data[['CodItem', 'Testing Cost']].head(10).sort_values(by = 'CodItem')

,CodItem,Testing Cost
3,70000103,0.80
9,70000168,2.38
4,70000224,1.02
8,70000392,1.43
1,70000638,2.46
6,70001206,2.20
2,70001597,2.11
5,70002653,1.29
0,70003237,1.78
7,70005903,2.20


In [95]:
transactional_data[['CodItem', 'Testing Cost']].head(10).sort_values(by = 'CodItem')

,CodItem,Testing Cost
3,70000103,"11,0"
9,70000168,"48,0"
4,70000224,"10,0"
8,70000392,"8,0"
1,70000638,"13,0"
6,70001206,"41,0"
2,70001597,"49,0"
5,70002653,"8,0"
0,70003237,"9,0"
7,70005903,"57,0"


os valores de testing cost estão diferentes para cada coditem em cada dataframe. na amostra observada, test_data, os valores de cada teste são sempre mais baixos

assim, é possível concluir que o testing cost no dataframe test_data é relativo ao custo do teste para a empresa, e no dataframe transactional_data é relativo ao valor que o cliente pagou pelo teste

é importante conferir se há valores duplicados na base test_data; se for o caso, há testes com mais de um valor

é importante também verificar se na base transactional_data há códigos de testes que não existem na base test_data, indicando que foram realizados testes que não tem seus valores tabelados

In [119]:
# contar o número de ocorrências de cada código na coluna CodItem da base test_data
n_cod = test_data['CodItem'].value_counts()

# filtrar os códigos que aparecem mais de uma vez
test_data[test_data['CodItem'] == n_cod[n_cod > 1].index.tolist()[0]]


,CodItem,Desc Item,Category,Family,Speciality,Testing Cost
1208,70004701,PROCEDIMENTO DIAGNOSTICO EM PAINEL DE IMUNOIST...,PA,ANATOMO,IMMUNOHISTOCHEMISTRY,362.96
1968,70004701,PROCEDIMENTO DIAGNOSTICO EM PAINEL DE IMUNOIST...,PA,ANATOMO,IMMUNOHISTOCHEMISTRY,362.96


aparentemente todas as informações deste teste estão duplicadas em todas as colunas

para fazer um double check, é possível confirmar que o Desc Item das colunas duplicadas não está escrito diferente em cada observação

In [121]:
test_data[test_data['CodItem'] == 70004701]['Desc Item'].tolist()

['PROCEDIMENTO DIAGNOSTICO EM PAINEL DE IMUNOISTOQUIMICA (DUAS A CINCO REACOES)',
 'PROCEDIMENTO DIAGNOSTICO EM PAINEL DE IMUNOISTOQUIMICA (DUAS A CINCO REACOES)']

as linhas estão de fato duplicadas. sendo assim, podemos remover a segunda observação

In [124]:
test_data2 = test_data['CodItem'].drop_duplicates()

# confirmando a remoção da linha duplicada

print('n linhas original:', len(test_data))
print('n linhas duplicado removido:', len(test_data2))

n linhas original: 2002
n linhas duplicado removido: 2001


agora, é importante ver se todos os testes presentes na base transactional_data estão tabelados na test_data

### tem muito mais observações na economic_data do que na demographic_data, por que isso acontece?

os dados na economic_data estão duplicados ou de fato, para alguns zip codes, temos informações econômicas mas não demográficas?

In [134]:
# verificando quantas vezes a base economic_data é maior que a demographic_data

len(economic_data) / len(demographic_data)

19.0

a base economic_data é exatamente 19 vezes maior que a demographic_data. investigar por que isso acontece

### todos os Geographic Area Name começam com ZCTA5?

se for este o caso, remover este código e deixar somente o número para facilitar o merge com a base df_geocode, que tem as informações dos laboratórios

transformar o nome da coluna para zipcode

### o formato dos dados das colunas Testing Cost das bases test_data e transactional_data está igual?

na preview das bases, um está com vírgula e outro com ponto

In [140]:
print(type(test_data['Testing Cost'].iloc[0]))
print(type(transactional_data['Testing Cost'].iloc[0]))

<class 'numpy.float64'>
<class 'str'>


mesmo que esta coluna tenha significados diferentes em cada base, é importante padronizar para que em ambas as bases, esta coluna esteja no formato float (número decimal)

ambas podem ser relacionadas para calcular o lucro, por exemplo

#### há missings nas bases??

fazer heatmaps para verificar se há valores faltantes